In [24]:
import pandas as pd
import os

def import_data():
    # Percorsi ai file JSON
    base_paths = [
        "data/automated-v1.0",
        "data/fake-v1.0"
    ]

    files = {
        "automated": "automatedAccountData.json",
        "nonautomated": "nonautomatedAccountData.json",
        "fake": "fakeAccountData.json",
        "real": "realAccountData.json"
    }

    # Costruisci i percorsi completi
    paths = {
        "automated": os.path.join(base_paths[0], files["automated"]),
        "nonautomated": os.path.join(base_paths[0], files["nonautomated"]),
        "fake": os.path.join(base_paths[1], files["fake"]),
        "real": os.path.join(base_paths[1], files["real"]),
    }

    # Leggi i file JSON come DataFrame
    df_automated = pd.read_json(paths["automated"])
    df_nonautomated = pd.read_json(paths["nonautomated"])
    df_fake = pd.read_json(paths["fake"])
    df_real = pd.read_json(paths["real"])

    # Unisci tutti i DataFrame in uno solo
    df_automated = pd.concat([df_automated, df_nonautomated], ignore_index=True)
    df_fake = pd.concat([df_fake, df_real], ignore_index=True)

    #print(f"\nTotale righe: {len(df_all)}")
    return df_automated, df_fake

In [25]:
df_automated, df_fake = import_data()

print(df_fake.head())

   userFollowerCount  userFollowingCount  userBiographyLength  userMediaCount  \
0                 25                1937                    0               0   
1                324                4122                    0               0   
2                 15                 399                    0               0   
3                 14                 107                    0               1   
4                264                4651                    0               0   

   userHasProfilPic  userIsPrivate  usernameDigitCount  usernameLength  isFake  
0                 1              1                   0              10       1  
1                 1              0                   4              15       1  
2                 0              0                   3              12       1  
3                 1              0                   1              10       1  
4                 1              0                   0              14       1  


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



# y = colonna target
y = df_fake["isFake"]

# X = tutte le colonne tranne 'isFake'
X = df_fake.drop("isFake", axis=1)

print(X[0:5])
print(y[0:5])


   userFollowerCount  userFollowingCount  userBiographyLength  userMediaCount  \
0                 25                1937                    0               0   
1                324                4122                    0               0   
2                 15                 399                    0               0   
3                 14                 107                    0               1   
4                264                4651                    0               0   

   userHasProfilPic  userIsPrivate  usernameDigitCount  usernameLength  
0                 1              1                   0              10  
1                 1              0                   4              15  
2                 0              0                   3              12  
3                 1              0                   1              10  
4                 1              0                   0              14  
0    1
1    1
2    1
3    1
4    1
Name: isFake, dtype: int64


In [29]:
# Suddividiamo in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2159407, shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Definizione del modello base
rf = RandomForestClassifier(random_state=2159407)

# Definizione della griglia dei parametri da ottimizzare
param_grid = {
    "n_estimators": [100, 200],           # numero di alberi
    "max_depth": [10, 20],           # profondità massima degli alberi
    "min_samples_split": [2, 5],           # minimo numero di campioni per split
    "min_samples_leaf": [1, 2],             # minimo numero di campioni per foglia
    "max_features": ["sqrt", "log2"]   # numero di feature da considerare per split
}

# GridSearch con 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                # 5-fold validation
    n_jobs=-1,           # usa tutti i core disponibili
    verbose=1,
    scoring='accuracy'   # possiamo usare anche 'f1' se il dataset è sbilanciato
)

# Fit del GridSearch sui dati di training
grid_search.fit(X_train, y_train)

# Parametri ottimali
print("Best parameters found:", grid_search.best_params_)

# Miglior modello
best_rf = grid_search.best_estimator_

# Valutazione sul test set
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuratezza sul test set: {accuracy:.2f}")
print("Report completo:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters found: {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Accuratezza sul test set: 0.95
